# Inequality constrains for background
In the notebook the inequality fit of the background will be shown on a single spectrum of SrTiO$_3$. To illustrate the problems that can arise with the non-constrained background, the fine structure is not incorporated in the model which is necessary if one wants to perform a good quantification of the EEL spectrum. For more information see the work of [Van den Broek et al](https://doi.org/10.1016/j.ultramic.2023.113830).


In [1]:
%matplotlib qt5

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pyEELSMODEL.api as em
from pyEELSMODEL.components.linear_background import LinearBG
from pyEELSMODEL.components.CLedge.zezhong_coreloss_edgecombined import ZezhongCoreLossEdgeCombined
from pyEELSMODEL.components.gdoslin import GDOSLin
from pyEELSMODEL.components.MScatter.mscatterfft import MscatterFFT

In [3]:
cdir = os.getcwd()
print('Current directory is: ' + cdir)

Current directory is: c:\Users\daen.jannis1\Documents\pyEELSMODEL\examples


In [4]:
filename = os.path.join(cdir, 'data', 'hl.msa')
filenamell = os.path.join(cdir, 'data', 'll.msa')

In [5]:
#The experimental parameters 
alpha = 0.9e-3 #convergence angle [rad]
beta = 2.675e-3 #collection angle [rad]
E0 = 300e3 #acceleration voltage [V]

In [6]:
# The edges in the spectrum
elements = ['Ti', 'O']
edges = ['L', 'K']

## Loading the datasets

In [7]:
s0 = em.Spectrum.load(filename)
ll = em.Spectrum.load(filenamell)

this will change the size and contents of the current spectrum
this will change the size and contents of the current spectrum


In [8]:
#the core loss spectrum
s0.plot()

In [9]:
#the low loss spectrum
ll.plot()

## Constrained fitting via Quadratic programming
A model based fitting approach is used to quantify the core-loss spectrum where three different constrains are applied to the backgroud.
1. No constrains, which is the ordinary least squares method
2. The parameters of the background are all positive
3. Inequality constrains on the background parameters to ensure convexity and a negative gradient. 


#### Model
The model  needs to be defined and consist out of four components.
1. Background: Linear background model, where the constrains can be modified
2. Titanium L edge
3. Oxygen K edge
4. Low-loss to take multiple scattering into account

In [10]:
def make_model(spectrum, elements, edges, constrains):
    specshape = s0.get_spectrumshape()
    BG = LinearBG(specshape, rlist=np.linspace(1,5,4))
    BG.use_approx = constrains
    
    comp_elements = []
    for elem, edge in zip(elements, edges):
        comp = ZezhongCoreLossEdgeCombined(specshape, 1, E0, alpha, beta, elem, edge)
        comp_elements.append(comp)

    ll_comp = MscatterFFT(specshape, ll)
    component_list = [BG]+comp_elements+[ll_comp]
    mod = em.Model(specshape, components=component_list) #The model object
    return mod

In [11]:
ols_model = make_model(s0, elements, edges, None) #model used for oridinary least squares
nnls_model = make_model(s0, elements, edges, 'non-neg') #model used for non-negative least squares
ineq_model = make_model(s0, elements, edges, 'sufficient') #model used for inequality constrained least squares

hll
hll
hll
hll
hll
hll
hll
hll
hll
hll
hll
hll


In [12]:
fit_ols = em.LinearFitter(s0, ols_model)
fit_nnls = em.QuadraticFitter(s0, nnls_model)
fit_ineq = em.QuadraticFitter(s0, ineq_model)
fits  = [fit_ols, fit_nnls, fit_ineq]
for fit in fits:
    fit.perform_fit()
    fit.plot()
    

cannot use analytical gradients since a convolutor is inside model
cannot use analytical gradients since a convolutor is inside model
cannot use analytical gradients since a convolutor is inside model


Visualization of the different background models obtained from the entire fit. It is clear that OLS has local maxima which is unphysical and the NNLS is not "flexible" enough to properly describe the background. In reality, the fine structure should be added to the model however this is done to illustrate problems that can occur for the OLS and NNLS methodologies.

In [13]:
fig, ax = plt.subplots()
mods = [ols_model, nnls_model, ineq_model]
lbs = ['OLS', 'NNLS', 'Ineq']
ax.plot(s0.energy_axis, s0.data, label='Data', color='black')
for jj, mod in enumerate(mods):
    ax.plot(s0.energy_axis, mod.components[0].data, label=lbs[jj])
ax.legend()